In [1]:
import matplotlib.pyplot as plt
import astropy
import os
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib as mpl
import pyarrow.parquet as pq
from astropy import io
from astropy.io import fits
from astropy import table
from astropy import wcs

In [2]:
catalogue_file = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Catalogue/tractor-0260m062.fits'

In [3]:
image_file_g = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/g-legacysurvey-0260m062-image.fits.fz'
image_file_r = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/r-legacysurvey-0260m062-image.fits.fz'
image_file_z = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/z-legacysurvey-0260m062-image.fits.fz'

In [4]:
catalogue = astropy.table.Table(astropy.io.fits.getdata(catalogue_file,ignore_missing_end=True))
catalogue

release,brickid,brickname,objid,brick_primary,brightblob,maskbits,type,ra,dec,ra_ivar,dec_ivar,bx,by,dchisq [5],ebv,mjd_min,mjd_max,ref_cat,ref_id,pmra,pmdec,parallax,pmra_ivar,pmdec_ivar,parallax_ivar,ref_epoch,gaia_pointsource,gaia_phot_g_mean_mag,gaia_phot_g_mean_flux_over_error,gaia_phot_g_n_obs,gaia_phot_bp_mean_mag,gaia_phot_bp_mean_flux_over_error,gaia_phot_bp_n_obs,gaia_phot_rp_mean_mag,gaia_phot_rp_mean_flux_over_error,gaia_phot_rp_n_obs,gaia_phot_variable_flag,gaia_astrometric_excess_noise,gaia_astrometric_excess_noise_sig,gaia_astrometric_n_obs_al,gaia_astrometric_n_good_obs_al,gaia_astrometric_weight_al,gaia_duplicated_source,gaia_a_g_val,gaia_e_bp_min_rp_val,gaia_phot_bp_rp_excess_factor,gaia_astrometric_sigma5d_max,gaia_astrometric_params_solved,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,fiberflux_g,fiberflux_r,fiberflux_z,fibertotflux_g,fibertotflux_r,fibertotflux_z,apflux_g [8],apflux_r [8],apflux_z [8],apflux_resid_g [8],apflux_resid_r [8],apflux_resid_z [8],apflux_ivar_g [8],apflux_ivar_r [8],apflux_ivar_z [8],mw_transmission_g,mw_transmission_r,mw_transmission_z,mw_transmission_w1,mw_transmission_w2,mw_transmission_w3,mw_transmission_w4,nobs_g,nobs_r,nobs_z,nobs_w1,nobs_w2,nobs_w3,nobs_w4,rchisq_g,rchisq_r,rchisq_z,rchisq_w1,rchisq_w2,rchisq_w3,rchisq_w4,fracflux_g,fracflux_r,fracflux_z,fracflux_w1,fracflux_w2,fracflux_w3,fracflux_w4,fracmasked_g,fracmasked_r,fracmasked_z,fracin_g,fracin_r,fracin_z,anymask_g,anymask_r,anymask_z,allmask_g,allmask_r,allmask_z,wisemask_w1,wisemask_w2,psfsize_g,psfsize_r,psfsize_z,psfdepth_g,psfdepth_r,psfdepth_z,galdepth_g,galdepth_r,galdepth_z,psfdepth_w1,psfdepth_w2,psfdepth_w3,psfdepth_w4,wise_coadd_id,lc_flux_w1 [11],lc_flux_w2 [11],lc_flux_ivar_w1 [11],lc_flux_ivar_w2 [11],lc_nobs_w1 [11],lc_nobs_w2 [11],lc_fracflux_w1 [11],lc_fracflux_w2 [11],lc_rchisq_w1 [11],lc_rchisq_w2 [11],lc_mjd_w1 [11],lc_mjd_w2 [11],fracdev,fracdev_ivar,shapeexp_r,shapeexp_r_ivar,shapeexp_e1,shapeexp_e1_ivar,shapeexp_e2,shapeexp_e2_ivar,shapedev_r,shapedev_r_ivar,shapedev_e1,shapedev_e1_ivar,shapedev_e2,shapedev_e2_ivar
int16,int32,str8,int32,bool,int16,int16,str4,float64,float64,float32,float32,float32,float32,float32,float32,float64,float64,str2,int64,float32,float32,float32,float32,float32,float32,float32,bool,float32,float32,int16,float32,float32,int16,float32,float32,int16,bool,float32,float32,int16,int16,float32,bool,float32,float32,float32,float32,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,uint8,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str8,float32,float32,float32,float32,int16,int16,float32,float32,float32,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
8000,294523,0260m062,0,True,0,0,DEV,25.94541155328336,-6.371715831454673,9123634000000.0,6916785300000.0,2811.9482,126.993515,546181.56 .. 787014.56,0.029117296,56576.2060457626,58094.11051504073,,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,False,0.0,0.0,0,0,0.0,False,0.0,0.0,0.0,0.0,0,14.647056,47.933273,105.962326,123.9659,84.309906,92.17993,226.64673,389.01837,196.1217,22.412415,1.6428819,0.3261016,0.0007680773,1.2045786e-05,3.291824,10.772672,23.814299,3.292312,10.773792,23.81583,1.2588413 .. 15.353642,4.7931805 .. 45.41997,11.696304 .. 91.22623,-0.10609738 .. 1.5384328,-0.08628096 .. 

In [5]:
image_z = astropy.table.Table(astropy.io.fits.getdata(image_file_z,ignore_missing_end=True))

In [6]:
dataframe = {}
for name in catalogue.colnames:
    data = catalogue[name].tolist()
    dataframe[name] = data

In [7]:
old_catalogue = pd.DataFrame(dataframe)

#old_catalogue

hdul = astropy.io.fits.open(image_file_g)
hdul[0].data
plt.figure(figsize = (12,12))
plt.imshow(hdul[0].data, cmap='hot')

hdul = astropy.io.fits.open(image_file_r)
hdul[0].data
plt.figure(figsize = (12,12))
plt.imshow(hdul[0].data, cmap='hot')

hdul = astropy.io.fits.open(image_file_z)
hdul[0].data
plt.figure(figsize = (12,12))
plt.imshow(hdul[0].data, cmap='hot')

In [8]:
original_image = image_file_z.split(os.path.sep)[-1]

#original_image

In [9]:
hdul = astropy.io.fits.open(image_file_z)

#hdul

In [10]:
w = astropy.wcs.WCS(hdul[0].header, naxis=2)

#w

In [11]:
x, y = w.wcs_world2pix(old_catalogue['ra'], old_catalogue['dec'], 1)

In [12]:
#x

In [13]:
#y

In [14]:
new_catalogue = pd.DataFrame()
new_catalogue['objid'] = catalogue['objid']
new_catalogue['original_image'] = [original_image] * len(new_catalogue)
new_catalogue['flux_g'] = catalogue['flux_g']
new_catalogue['flux_r'] = catalogue['flux_r']
new_catalogue['flux_z'] = catalogue['flux_z']
new_catalogue['x'] = x
new_catalogue['y'] = y
new_catalogue['ra'] = catalogue['ra']
new_catalogue['dec'] = catalogue['dec']

#new_catalogue

In [15]:
def convert_tractor_catalogue(catalogue_file, image_file, image_name=''):
    """
    Converts a tractor fits file to a pandas dataframe to be given
    directly to an ImageDataset object.

    Parameters
    ----------
    catalogue_files : string
        tractor catalogue in fits table format 
    image_file:
        The image corresponding to this catalogue (to extract pixel information
        and naming information)
    """

    catalogue = astropy.table.Table(astropy.io.fits.getdata(catalogue_file,ignore_missing_end=True))

    dataframe = {}
    for name in catalogue.colnames:
        data = catalogue[name].tolist()
        dataframe[name] = data
    
    old_catalogue = pd.DataFrame(dataframe)
    hdul = astropy.io.fits.open(image_file)

    if len(image_name) == 0:
        original_image = image_file.split(os.path.sep)[-1]
    else:
        original_image = image_name
    
    #w = astropy.wcs.WCS(hdul[0].header, naxis=2)
    #x, y = w.wcs_world2pix(old_catalogue['ra'], old_catalogue['dec'], 1)
    
    new_catalogue = pd.DataFrame()
    new_catalogue['objid'] = old_catalogue['objid']
    new_catalogue['original_image'] = [original_image] * len(new_catalogue)
    new_catalogue['flux_g'] = old_catalogue['flux_g']
    new_catalogue['flux_r'] = old_catalogue['flux_r']
    new_catalogue['flux_z'] = old_catalogue['flux_z']
    new_catalogue['x'] = old_catalogue['bx'].astype('int')
    new_catalogue['y'] = old_catalogue['by'].astype('int')
    new_catalogue['ra'] = old_catalogue['ra']
    new_catalogue['dec'] = old_catalogue['dec']
    
    new_catalogue['type'] = old_catalogue['type']
    
    return new_catalogue

In [16]:
image_name = 'legacysurvey-0260m062-image.fits.fz'

In [17]:
prod = convert_tractor_catalogue(catalogue_file, image_file_z, image_name)

In [18]:
new_catalogue.loc[66]

objid                                                66
original_image    z-legacysurvey-0260m062-image.fits.fz
flux_g                                          16.3139
flux_r                                          29.5412
flux_z                                          44.2382
x                                               26.0834
y                                               -6.3683
ra                                              26.0834
dec                                             -6.3683
Name: 66, dtype: object

In [19]:
image_z

<Table length=3600>
     col0          col1           col2     ...    col3598        col3599   
   float32       float32        float32    ...    float32        float32   
------------- -------------- ------------- ... -------------- -------------
  0.012542456    0.001042011  -0.012241758 ...   -0.004113502  -0.006427368
 0.0053520175   -0.005064261  -0.016058298 ...  -0.0017288908  0.0057287705
 -0.005442136  -0.0036008838   0.006325477 ...   0.0033504243 -0.0033193938
 -0.004290125  -0.0046543176   0.011562827 ...    0.022130104  0.0075719752
 -0.006568665   0.0153176095   0.010241485 ...  -0.0006315416   0.011977259
-0.0006025392    0.014301952 -0.0038046953 ...   -0.002430924  0.0068547945
 0.0005893898  -0.0057469835  0.0006264823 ...    0.004516727 -7.033924e-05
 -0.010779938    0.017766183   0.012240775 ... -0.00029440306  -0.016435593
  0.005720138  -0.0014558281  0.0017750456 ...   0.0041110646    0.01386351
 0.0016454022    0.004653602   0.008271941 ...    0.007998779   0.009390121
          ...            ...           ... ...            ...           ...
-0.0052127223    0.018174509  -0.008529916 ...    0.011994438  -0.005465188
 -0.007015211  -0.0035580867   -0.01746464 ...   0.0020709955   -0.00437742
 0.0063621667  -0.0075649177   0.011123213 ...    0.012337938  0.0014366122
 0.0007321214   -0.007937604  0.0024918441 ...    0.011946907  -0.005723971
-0.0012213282    0.004735507   0.004064629 ...   -0.011924889   0.009290435
-0.0012931883  -0.0014187705  0.0054219654 ...   -0.019012239 -0.0024989326
  0.013780691  0.00027917328 -0.0021361005 ...    0.013103647   0.009508343
 0.0026569986  -0.0047268355   0.010268128 ...   -0.010340031    0.00158461
  0.014805317  -0.0015301802  0.0018576744 ...  0.00018430085   0.010645507
 -0.007282085 -0.00060299796 -0.0012881856 ...     0.01084272  0.0032035515

### Remove Point Sources

In [20]:
catalogue_no_PSFs = prod[prod.type.astype("S") != b'PSF ']

In [21]:
catalogue_no_PSFs.sort_values('flux_r', ascending=False)

,objid,original_image,flux_g,flux_r,flux_z,x,y,ra,dec,type
594,594,legacysurvey-0260m062-image.fits.fz,317.535095,720.682556,1307.191528,337,369,26.126581,-6.354089,DEV
6095,6095,legacysurvey-0260m062-image.fits.fz,168.212524,284.601593,412.424927,994,2486,26.078486,-6.199997,EXP
2709,2709,legacysurvey-0260m062-image.fits.fz,51.362320,146.547363,276.822815,3216,1209,25.915781,-6.292911,DEV
5667,5667,legacysurvey-0260m062-image.fits.fz,71.172943,143.411240,247.036163,1088,2359,26.071583,-6.209265,EXP
2471,2471,legacysurvey-0260m062-image.fits.fz,43.115738,121.118385,238.656158,2859,1137,25.941948,-6.298205,DEV
...,...,...,...,...,...,...,...,...,...,...
1387,1387,legacysurvey-0260m062-image.fits.fz,-0.002538,0.027460,0.509285,3125,619,25.922422,-6.335852,REX
4825,4825,legacysurvey-0260m062-image.fits.fz,0.005111,0.024595,0.499707,1270,1938,26.058274,-6.239884,REX
8559,8559,legacysurvey-0260m062-image.fits.fz,0.000000,0.000000,0.000000,337,369,26.126580,-6.354087,DUP
2941,2941,legacysurvey-0260m062-image.fits.fz,0.109954,-0.004085,0.056334,1629,1271,26.032034,-6.288446,REX


### 500 Brightest objects

In [22]:
test_catalogue_0260m062_500 = catalogue_no_PSFs.sort_values('flux_z', ascending=False)[0:500]

test_catalogue_0260m062_500

,objid,original_image,flux_g,flux_r,flux_z,x,y,ra,dec,type
594,594,legacysurvey-0260m062-image.fits.fz,317.535095,720.682556,1307.191528,337,369,26.126581,-6.354089,DEV
6095,6095,legacysurvey-0260m062-image.fits.fz,168.212524,284.601593,412.424927,994,2486,26.078486,-6.199997,EXP
2709,2709,legacysurvey-0260m062-image.fits.fz,51.362320,146.547363,276.822815,3216,1209,25.915781,-6.292911,DEV
5667,5667,legacysurvey-0260m062-image.fits.fz,71.172943,143.411240,247.036163,1088,2359,26.071583,-6.209265,EXP
2471,2471,legacysurvey-0260m062-image.fits.fz,43.115738,121.118385,238.656158,2859,1137,25.941948,-6.298205,DEV
...,...,...,...,...,...,...,...,...,...,...
6345,6345,legacysurvey-0260m062-image.fits.fz,0.981929,2.535009,4.758138,292,2585,26.129869,-6.192759,REX
4612,4612,legacysurvey-0260m062-image.fits.fz,1.468138,3.162834,4.752346,3450,1864,25.898683,-6.245235,EXP
2445,2445,legacysurvey-0260m062-image.fits.fz,0.865858,2.835724,4.745888,1230,1161,26.061230,-6.296397,DEV
3190,3190,legacysurvey-0260m062-image.fits.fz,0.940797,2.619087,4.742189,2916,1365,25.937799,-6.281606,EXP


## Results

In [23]:
test_catalogue_0260m062_500.to_csv ('/home/verlon/Desktop/Astronomaly/Data/test_catalogue_0260m062_500.csv', index = True, header=True)

In [24]:
interesting_objects = pd.DataFrame()
interesting_objects['x'] = [151,2289,1717,461,2499,994,3216,332,1905,306,620,646,412]  #x = 412, y = 2059 is possibly a lens!
interesting_objects['y'] = [994,1858,667,3391,215,2486,1209,2126,3376,1364,1426,1372,2059]

In [25]:
TSNE_Plot = pq.read_table('/home/verlon/Desktop/Astronomaly/Data/Output/Coadd026/0260m062_500_test/TSNE_Plot_output.parquet').to_pandas()

### Undetected objects

In [26]:
undetected = test_catalogue_0260m062_500[~test_catalogue_0260m062_500.index.isin(TSNE_Plot.index)]

In [27]:
undetected.sort_values('flux_z', ascending=False)[0:10]

,objid,original_image,flux_g,flux_r,flux_z,x,y,ra,dec,type
66,66,legacysurvey-0260m062-image.fits.fz,16.313875,29.541180,44.238152,927,173,26.083418,-6.368302,EXP
6579,6579,legacysurvey-0260m062-image.fits.fz,3.275764,11.530080,25.820513,2141,2697,25.994492,-6.184672,DEV
565,565,legacysurvey-0260m062-image.fits.fz,8.224695,15.305624,24.295431,331,424,26.127054,-6.350033,EXP
1361,1361,legacysurvey-0260m062-image.fits.fz,1.664122,8.564999,23.716200,441,623,26.118971,-6.335610,DEV
6553,6553,legacysurvey-0260m062-image.fits.fz,2.579007,9.657364,19.260395,2357,2654,25.978684,-6.187744,DEV
1337,1337,legacysurvey-0260m062-image.fits.fz,2.340322,7.369914,16.031267,3194,612,25.917372,-6.336376,REX
575,575,legacysurvey-0260m062-image.fits.fz,5.294921,10.997431,15.493241,314,303,26.128289,-6.358874,EXP
4409,4409,legacysurvey-0260m062-image.fits.fz,2.993342,7.398978,15.010366,2924,1755,25.937180,-6.253220,EXP
6074,6074,legacysurvey-0260m062-image.fits.fz,1.376793,6.610613,14.977666,588,2486,26.108228,-6.199971,REX
8102,8102,legacysurvey-0260m062-image.fits.fz,0.754538,4.525452,14.517697,1386,3315,26.049752,-6.139646,DEV
